In [1]:
API_KEY = '67c1b98b5edbb9d798ac60a8d13f271e'
USER_AGENT = 'Project'

In [2]:
import requests

headers = {
    'user-agent': USER_AGENT
}

payload = {
    'api_key': API_KEY,
    'method': 'chart.gettopartists',
    'format': 'json'
}

r = requests.get('https://ws.audioscrobbler.com/2.0/', headers=headers, params=payload)
r.status_code

200

In [3]:
def lastfm_get(payload):
    # define headers and URL
    headers = {'user-agent': USER_AGENT}
    url = 'https://ws.audioscrobbler.com/2.0/'

    # Add API key and format to the payload
    payload['api_key'] = API_KEY
    payload['format'] = 'json'

    response = requests.get(url, headers=headers, params=payload)
    return response

In [4]:
r = lastfm_get({
    'method': 'chart.gettopartists'
})

r.status_code

200

In [5]:
import json

def jprint(obj):
    # create a formatted string of the Python JSON object
    text = json.dumps(obj, sort_keys=True, indent=4)
    print(text)

jprint(r.json())

{
    "artists": {
        "@attr": {
            "page": "1",
            "perPage": "50",
            "total": "5851377",
            "totalPages": "117028"
        },
        "artist": [
            {
                "image": [
                    {
                        "#text": "https://lastfm.freetls.fastly.net/i/u/34s/2a96cbd8b46e442fc41c2b86b821562f.png",
                        "size": "small"
                    },
                    {
                        "#text": "https://lastfm.freetls.fastly.net/i/u/64s/2a96cbd8b46e442fc41c2b86b821562f.png",
                        "size": "medium"
                    },
                    {
                        "#text": "https://lastfm.freetls.fastly.net/i/u/174s/2a96cbd8b46e442fc41c2b86b821562f.png",
                        "size": "large"
                    },
                    {
                        "#text": "https://lastfm.freetls.fastly.net/i/u/300x300/2a96cbd8b46e442fc41c2b86b821562f.png",
                        "s

In [6]:
def fetch_user_info(user):
    """
    Fetches user information for a given user from the Last.fm API.
    """
    try:
        # Make the API request with the user passed as a parameter
        response1 = lastfm_get({
            'method': 'user.getinfo',
            'user': user
        })

        # Check if the request was successful
        if response1.status_code == 200:
            # If successful, return the JSON data
            return response1.json()
        else:
            # If unsuccessful, print an error message
            print(f"Failed to fetch info for user '{user}'. Status code: {response1.status_code}, Response: {response1.text}")
            return None

    except requests.exceptions.RequestException as e:
        # Handle any exceptions raised during the request
        print(f"Exception occurred while fetching info for user '{user}': {e}")
        return None

In [7]:

def fetch_user_friends(user):
    """
    Fetches the friends of a given user from the Last.fm API.
    """
    try:
        # Make the API request with the user parameter
        response2 = lastfm_get({
            'method': 'user.getFriends',
            'user': user
        })

        # Check if the request was successful
        if response2.status_code == 200:
            # If successful, return the JSON data
            return response2.json()
        else:
            # If unsuccessful, print an error message with the response body for debugging
            print(f"Failed to fetch friends for user '{user}'. Status code: {response2.status_code}, Response: {response2.text}")
            return None

    except requests.exceptions.RequestException as e:
        # Handle any exceptions raised during the request (e.g., network issues)
        print(f"Exception occurred while fetching friends for user '{user}': {e}")
        return None


In [8]:
############################## This Function Fetches a list of users without duplicates ##################################

import pandas as pd
from collections import deque
import time

def get_friends_of_user(user):
    """
    Uses the Last.fm API to get the friends of a given user.
    """
    response = lastfm_get({
        'method': 'user.getFriends',
        'user': user
    })

    # Check if the request was successful
    if response.status_code == 200:
        data = response.json()
        friends = [friend['name'] for friend in data.get('friends', {}).get('user', [])]
        return friends
    else:
        print(f"Failed to fetch friends for user {user}. Status code: {response.status_code}")
        return []

def build_user_friend_network(start_user, max_levels=2):
    """
    Builds a user-friend network by iterating through friends of each user.
    """
    processed_users = set()  # Track processed users to avoid repetition
    user_queue = deque([(start_user, 0)])  # Queue for BFS with level tracking
    all_users = {}  # Dictionary to store user: friends list

    while user_queue:
        current_user, level = user_queue.popleft()

        # Stop if we've reached the maximum level of depth
        if level > max_levels:
            break

        # Skip if we've already processed this user
        if current_user in processed_users:
            continue

        # Mark this user as processed
        processed_users.add(current_user)

        # Fetch the friends of the current user
        friends = get_friends_of_user(current_user)
        all_users[current_user] = friends  # Store the friends list for this user

        # Add each friend to the queue for further processing, if not already processed
        for friend in friends:
            if friend not in processed_users:
                user_queue.append((friend, level + 1))

        # Optional: add delay to respect API rate limits
        time.sleep(0.1)  # Adjust as needed

    # Convert the dictionary to a DataFrame for analysis or storage
    df = pd.DataFrame([(user, friend) for user, friends in all_users.items() for friend in friends], columns=["user", "Friend"])
    df.drop('user', axis=1, inplace=True)
    # Get unique friends list
    friends_list = df['Friend'].tolist()
    unique_friends = list(dict.fromkeys(friends_list))
    df2 = pd.DataFrame(unique_friends, columns=["Users"])



    # Save the unique friends list to CSV and Excel
    csv_filename = "users_without_deuplicates.csv"
    df2.to_csv(csv_filename, index=False)
    print(f"Saved friends list to {csv_filename}")

    excel_filename = "user_without_duplicates.xlsx"
    df2.to_excel(excel_filename, index=False)
    print(f"Saved friends list to {excel_filename}")

    return unique_friends



In [9]:
def fetch_loved_tracks(user):
    """
    Fetches the loved tracks of a given user from the Last.fm API.
    """
    try:
        # Make the API request with the specified user parameter
        response3 = lastfm_get({
            'method': 'user.getLovedTracks',
            'user': user
        })

        # Check if the request was successful
        if response3.status_code == 200:
            # If successful, save the JSON data to a variable and return it
            return response3.json()
        else:
            # If unsuccessful, print an error message with the response body for debugging
            print(f"Failed to fetch loved tracks for user '{user}'. Status code: {response3.status_code}, Response: {response3.text}")
            return None

    except requests.exceptions.RequestException as e:
        # Handle any exceptions raised during the request (e.g., network issues)
        print(f"Exception occurred while fetching loved tracks for user '{user}': {e}")
        return None


In [10]:
def fetch_personal_tags(user, tag):
    """
    Fetches the personal tags for a specific tag and tagging type ('track') of a given user.
    """
    try:
        # Make the API request with user, tag, and tagging_type parameters
        response4 = lastfm_get({
            'method': 'user.getPersonalTags',
            'user': user,
            'tag': tag,
            'taggingtype': 'track'
        })

        # Check if the request was successful
        if response4.status_code == 200:
            # If successful, save the JSON data to a variable and return it
            return response4.json()
        else:
            # If unsuccessful, print an error message with the response body for debugging
            print(f"Failed to fetch personal tags for user '{user}' with tag '{tag}'. "
                  f"Status code: {response4.status_code}, Response: {response4.text}")
            return None

    except requests.exceptions.RequestException as e:
        # Handle any exceptions raised during the request (e.g., network issues)
        print(f"Exception occurred while fetching personal tags for user '{user}' with tag '{tag}': {e}")
        return None


In [11]:
def fetch_recent_tracks(user):
    """
    Fetches the recent tracks of a given user.
    """
    try:
        # Make the API request with the user parameter
        response5 = lastfm_get({
            'method': 'user.getRecentTracks',
            'user': user
        })

        # Check if the request was successful
        if response5.status_code == 200:
            # If successful, save the JSON data to a variable and return it
            return response5.json()
        else:
            # If unsuccessful, print an error message with the response body for debugging
            print(f"Failed to fetch recent tracks for user '{user}'. "
                  f"Status code: {response5.status_code}, Response: {response5.text}")
            return None

    except requests.exceptions.RequestException as e:
        # Handle any exceptions raised during the request (e.g., network issues)
        print(f"Exception occurred while fetching recent tracks for user '{user}': {e}")
        return None


In [12]:

def fetch_top_albums(user):
    """
    Fetches the top albums for a given user.
    """
    try:
        # Make the API request with the user parameter
        response6 = lastfm_get({
            'method': 'user.getTopAlbums',
            'user': user
        })

        # Check if the request was successful
        if response6.status_code == 200:
            # If successful, return the JSON data
            return response6.json()

        else:
            # If unsuccessful, log the error along with the response body for debugging
            print(f"Failed to fetch top albums for user '{user}'. "
                  f"Status code: {response6.status_code}, Response: {response6.text}")
            return None

    except requests.exceptions.RequestException as e:
        # Handle any exceptions raised during the request (e.g., network issues)
        print(f"Exception occurred while fetching top albums for user '{user}': {e}")
        return None


In [13]:

def fetch_top_artists(user):
    """
    Fetches the top artists of a given user.
    """
    try:
        # Make the API request with the user parameter
        response7 = lastfm_get({
            'method': 'user.getTopArtists',
            'user': user
        })

        # Check if the request was successful
        if response7.status_code == 200:
            # If successful, return the JSON data
            return response7.json()

        else:
            # If unsuccessful, log the error with the status code and response body
            print(f"Failed to fetch top artists for user '{user}'. "
                  f"Status code: {response7.status_code}, Response: {response7.text}")
            return None

    except requests.exceptions.RequestException as e:
        # Handle any exceptions raised during the request (e.g., network issues)
        print(f"Exception occurred while fetching top artists for user '{user}': {e}")
        return None


In [14]:

def fetch_top_tags(user):
    """
    Fetches the top tags of a given user with exception handling.
    """
    try:
        # Make the API request with the user parameter
        response8 = lastfm_get({
            'method': 'user.getTopTags',
            'user': user
        })

        # Check if the request was successful
        if response8.status_code == 200:
            # If successful, save the JSON data to a variable
            return response8.json()
        else:
            # If unsuccessful, print an error message
            print(f"Failed to fetch top tags for user '{user}'. Status code: {response8.status_code}")
            return None

    except requests.exceptions.RequestException as e:
        # Handle any exceptions raised during the request (e.g., network issues)
        print(f"Exception occurred while fetching top tags for user '{user}': {e}")
        return None


In [15]:
def fetch_top_tracks(user):
    """
    Fetches the top tracks of a given user with detailed exception handling.
    """
    try:
        # Make the API request with the user parameter
        response9 = lastfm_get({
            'method': 'user.getTopTracks',
            'user': user
        })

        # Check if the request was successful
        if response9.status_code == 200:
            # If successful, return the JSON data
            return response9.json()
        else:
            # If unsuccessful, print an error message
            print(f"Failed to fetch top tracks for user '{user}'. Status code: {response9.status_code}")
            return None

    except requests.exceptions.RequestException as e:
        # Handle any network-related exceptions
        print(f"Exception occurred while fetching top tracks for user '{user}': {e}")
        return None

    except Exception as e:
        # Catch any other unexpected exceptions
        print(f"An unexpected error occurred while fetching top tracks for user '{user}': {e}")
        return None


In [16]:


def fetch_weekly_album_chart(user):
    """
    Fetches the weekly album chart for a given user within a specified date range.
    The date range should be passed as datetime objects.
    """
    try:

        # Make the API request with the user and date range
        response10 = lastfm_get({
            'method': 'user.getWeeklyAlbumChart',
            'user': user,
            "from": "1212321600",
            "to":   "1212926400"
        })

        # Check if the request was successful
        if response10.status_code == 200:
            # If successful, return the JSON data
            return response10.json()
        else:
            # If unsuccessful, log the error with status code and response body
            print(f"Failed to fetch weekly album chart for user {user}. "
                  f"Status code: {response10.status_code}, Response: {response10.text}")
            return None

    except requests.exceptions.RequestException as e:
        # Handle any exceptions raised during the request (e.g., network issues)
        print(f"Exception occurred while fetching weekly album chart for user '{user}': {e}")
        return None


In [17]:


def fetch_weekly_artist_chart(user):
    """
    Fetches the weekly artist chart for a given user within a specified date range.
    The date range should be passed as datetime objects.
    """
    try:


        # Make the API request with the user and date range
        response11 = lastfm_get({
            'method': 'user.getWeeklyArtistChart',
            'user': user,
            "from": "1212321600",
            "to": "1212926400"
        })

        # Check if the request was successful
        if response11.status_code == 200:
            # If successful, return the JSON data
            return response11.json()
        else:
            # If unsuccessful, log the error with status code and response body
            print(f"Failed to fetch weekly artist chart for user {user}. "
                  f"Status code: {response11.status_code}, Response: {response11.text}")
            return None

    except requests.exceptions.RequestException as e:
        # Handle any exceptions raised during the request (e.g., network issues)
        print(f"Exception occurred while fetching weekly artist chart for user '{user}': {e}")
        return None


In [18]:

def fetch_weekly_chart_list(user):
    """
    Fetches the weekly chart list for a given user.
    """
    try:
        # Make the API request with the user parameter
        response12 = lastfm_get({
            'method': 'user.getWeeklyChartList',
            'user': user
        })

        # Check if the request was successful (status code 200)
        if response12.status_code == 200:
            # If successful, return the JSON data
            return response12.json()
        else:
            # If unsuccessful, log the error with status code and response body
            print(f"Failed to fetch weekly chart list for user {user}. "
                  f"Status code: {response12.status_code}, Response: {response12.text}")
            return None
    except requests.exceptions.RequestException as e:
        # Handle any exceptions raised during the request (e.g., network issues)
        print(f"Exception occurred while fetching weekly chart list for user '{user}': {e}")
        return None


In [19]:

def fetch_user_weekly_track_chart(user):
    """
    Fetches the weekly track chart for a given user and stores the response in a variable.
    """
    try:
        # Make the API request with the user and time parameters
        response13 = lastfm_get({
            'method': 'user.getWeeklyTrackChart',
            'user': user,
            "from": "1212321600",  # Start date in Unix timestamp
            "to": "1212926400"     # End date in Unix timestamp
        })

        # Check if the request was successful (status code 200)
        if response13.status_code == 200:
            # Store the JSON response in a variable and return it
            return response13.json()
        else:
            # If unsuccessful, log the error with status code and response body
            print(f"Failed to fetch weekly track chart for user {user}. "
                  f"Status code: {response13.status_code}, Response: {response13.text}")
            return None
    except requests.exceptions.RequestException as e:
        # Handle any exceptions raised during the request (e.g., network issues)
        print(f"Exception occurred while fetching weekly track chart for user '{user}': {e}")
        return None


In [24]:
import pandas as pd
def fetch_user_data(user):
    """
    Fetch the data for a user from various API methods
    and store the responses in a dictionary.
    """
    user_info = fetch_user_info(user)
    user_friends = fetch_user_friends(user)
    loved_tracks = fetch_loved_tracks(user)
    personal_tags1 = fetch_personal_tags(user, "rock")
    personal_tags2= fetch_personal_tags(user, "electronic")
    personal_tags3 = fetch_personal_tags(user, "seen live")
    personal_tags4 = fetch_personal_tags(user, "alternative")
    personal_tags5 = fetch_personal_tags(user, "indie")
    personal_tags6= fetch_personal_tags(user, "pop")

    recent_tracks = fetch_recent_tracks(user)
    top_albums = fetch_top_albums(user)
    top_artists = fetch_top_artists(user)
    top_tags = fetch_top_tags(user)
    top_tracks = fetch_top_tracks(user)
    weeklyalbumchart=fetch_weekly_album_chart(user)
    weeklyartistchart=fetch_weekly_artist_chart(user)
    weeklytrackchart= fetch_user_weekly_track_chart(user)



    final = {
    "user_info": user_info,
    "user_friends": user_friends,
    "loved_tracks": loved_tracks,
    "personal_tags1":personal_tags1,
    "personal_tags2":personal_tags2,
    "personal_tags3":personal_tags3,
    "personal_tags4":personal_tags4,
    "personal_tags5":personal_tags5,
    "personal_tags6":personal_tags6,
    "recet_tracks" :recent_tracks,
    "top_albums":top_albums,
    "top_artists":top_artists,
    "top_tags":top_tags,
    "top_tracks":top_tracks,
    "weekly_album_chart":weeklyalbumchart,
    "weekly_artist_chart":weeklyartistchart,
    "weekly_track_chart":weeklytrackchart,
    #"Geography":country,
    # "Geo":top_tracks_India


}

    return final


def extract_user_details(final):
   # Ensure each step returns a valid dictionary or handle it accordingly
 user_info = final.get('user_info', None)
 if user_info and isinstance(user_info, dict):
    user = user_info.get('user', None)
    if user and isinstance(user, dict):
        name = user.get('name', None)
        realname = user.get('realname', None)
        age = user.get('age', None)
        gender = user.get('gender', None)
        country = user.get('country', None)
        album_count = user.get('album_count', None)
        artist_count = user.get('artist_count', None)
        playlist = user.get('playlists', None)
        subscriber = user.get('subscriber', None)
    else:
        # If 'user' is not a dictionary or does not exist
        name = realname = age = gender = country = album_count = artist_count = playlist = subscriber = None
 else:
    # If 'user_info' is not a dictionary or does not exist
  name = realname = age = gender = country = album_count = artist_count = playlist = subscriber = None


    # Collecting lists from nested structures
 friends = [
    friend.get('name') for friend in (final.get('user_friends') or {}).get('friends', {}).get('user', [])
    if isinstance(friend, dict) and 'name' in friend
    ]
 loved_tracks = [
    track.get('name') for track in (final.get('loved_tracks') or {}).get('lovedtracks', {}).get('track', [])
    if isinstance(track, dict) and 'name' in track  # Ensure 'name' exists in each track dictionary
    ]

    # Extract rock_tracks
 rock_tracks = [
    track.get('name') for track in (final.get('personal_tags1') or {}).get('taggings', {}).get('tracks', {}).get('track', [])
    if isinstance(track, dict) and 'name' in track
    ]
 electronic_tracks = [
    track.get('name') for track in (final.get('personal_tags2') or {}).get('taggings', {}).get('tracks', {}).get('track', [])
    if isinstance(track, dict) and 'name' in track
    ]
 seen_live_tracks = [
    track.get('name') for track in (final.get('personal_tags3') or {}).get('taggings', {}).get('tracks', {}).get('track', [])
    if isinstance(track, dict) and 'name' in track
    ]
 alternative_tracks = [
    track.get('name') for track in (final.get('personal_tags4') or {}).get('taggings', {}).get('tracks', {}).get('track', [])
    if isinstance(track, dict) and 'name' in track
    ]
 indie_tracks = [
    track.get('name') for track in (final.get('personal_tags5') or {}).get('taggings', {}).get('tracks', {}).get('track', [])
    if isinstance(track, dict) and 'name' in track
    ]
 pop_tracks = [
    track.get('name') for track in (final.get('personal_tags6') or {}).get('taggings', {}).get('tracks', {}).get('track', [])
    if isinstance(track, dict) and 'name' in track
    ]
    # Extract recent_tracks
 recent_tracks = [
    track.get('name') for track in (final.get('recent_tracks') or {}).get('recenttracks', {}).get('track', [])
    if isinstance(track, dict) and 'name' in track
    ]

    # Extract top_albums
 top_albums = [
    album.get('name') for album in (final.get('top_albums') or {}).get('topalbums', {}).get('album', [])
    if isinstance(album, dict) and 'name' in album
    ]

    # Extract top_artists
 top_artists = [
    artist.get('name') for artist in (final.get('top_artists') or {}).get('topartists', {}).get('artist', [])
    if isinstance(artist, dict) and 'name' in artist
    ]

    # Extract top_tags
 top_tags = [
    tag.get('name') for tag in (final.get('top_tags') or {}).get('toptags', {}).get('tag', [])
    if isinstance(tag, dict) and 'name' in tag
    ]

    # Extract top_tracks
 top_tracks = [
    track.get('name') for track in (final.get('top_tracks') or {}).get('toptracks', {}).get('track', [])
    if isinstance(track, dict) and 'name' in track
    ]


    # Create a DataFrame
 dfin1 = pd.DataFrame({
        'Name': [name],
        'RealName': [realname],
        'Age': [age],
        'Gender': [gender],
        'Country': [country],
        'Album_Count': [album_count],
        'Artist_Count': [artist_count],
        'Playlist': [playlist],
        'Subscriber': [subscriber],
        'Friends': [friends],
        'loved_Tracks': [loved_tracks],
        'Rock_Tracks': [rock_tracks],
        'electronic_Tracks': [electronic_tracks],
        'seen_live_Tracks': [seen_live_tracks],
        'alternative_Tracks': [alternative_tracks],
        'indie_Tracks': [indie_tracks],
        'pop_Tracks': [pop_tracks],
        'Recent_Tracks': [recent_tracks],
        'Top_Albums': [top_albums],
        'Top_Artists': [top_artists],
        'Top_Tags': [top_tags],
        'Top_Tracks': [top_tracks]
    })


 return dfin1

def fetch_and_extract(user):
    """Fetches user data and extracts details for a single user."""
    final = fetch_user_data(user)
    return extract_user_details(final)

def process_all_friends():
    frandsd = build_user_friend_network("RJ")  # Full list of friends (30,000 users)
    frands= frandsd[:5000]
    batch_size = 100  # Adjust batch size as needed
    all_friends_df = []

    # Divide the list into batches
    batches = [frands[i:i + batch_size] for i in range(0, len(frands), batch_size)]

    for batch in batches:
        # Process each user sequentially in the batch
        for user in batch:
            try:
                # Fetch and extract user details
                friend_df = fetch_and_extract(user)
                all_friends_df.append(friend_df)
            except Exception as e:
                print(f"Error processing user {user}: {e}")

    # Combine all individual user DataFrames into one large DataFrame
    all_friends_df_combined = pd.concat(all_friends_df, ignore_index=True)

    # Save the combined DataFrame as CSV or Excel
    all_friends_df_combined.to_csv("all_friends_details.csv", index=False)
    all_friends_df_combined.to_excel("all_friends_details.xlsx", index=False)

    print("All friends details have been saved to 'all_friends_details.csv' and 'all_friends_details.xlsx'")
    return all_friends_df_combined

In [25]:
process_all_friends()

Failed to fetch friends for user BBCRadio3. Status code: 400
Failed to fetch friends for user sajones96. Status code: 400
Failed to fetch friends for user Irina4778. Status code: 400
Failed to fetch friends for user FvckSwag. Status code: 404
Failed to fetch friends for user TeslasWife. Status code: 404
Failed to fetch friends for user Oughav. Status code: 404
Failed to fetch friends for user Zeritas. Status code: 404
Failed to fetch friends for user Amour_Noir. Status code: 404
Failed to fetch friends for user bigoldoinks. Status code: 400
Failed to fetch friends for user fornotwo. Status code: 404
Failed to fetch friends for user Rausty. Status code: 404
Failed to fetch friends for user anona_. Status code: 400
Failed to fetch friends for user koolherc11111. Status code: 400
Failed to fetch friends for user francis_bone. Status code: 400
Failed to fetch friends for user ra_rigg. Status code: 400
Failed to fetch friends for user jvhyny. Status code: 400
Failed to fetch friends for use

,Name,RealName,Age,Gender,Country,Album_Count,Artist_Count,Playlist,Subscriber,Friends,...,electronic_Tracks,seen_live_Tracks,alternative_Tracks,indie_Tracks,pop_Tracks,Recent_Tracks,Top_Albums,Top_Artists,Top_Tags,Top_Tracks
0,Babs_05,Badass Mod from the Future - Last Seen Tomorro...,0,n,None,56949,25434,0,0,"[GlennyGlen, tapenoon, bbc1xtra, BBCRadio3, sp...",...,"[Dark & Long, The White Flash (feat. Thom York...",[],"[Requiem For Anna, Impossible Things #2, M'aid...","[The Cold Swedish Winter, Impossible Things #2...","[Music/Impressive Instant, I Don't Like The Vi...",[],"[Tomboy, The Slow Rush, Slow Focus, Within and...","[Tame Impala, Radiohead, Panda Bear, Beach Hou...","[my gang 09, music to fall asleep to, chillout...","[Severance, The Red Wing, Year Of The Dog, One..."
1,franhale,Fran,0,n,United Kingdom,11534,5684,0,0,"[rossmillican, bhavnit, tburny, protemaziclon,...",...,[],[],[],[],[],[],"[Nimrod, Split The Atom: Special Edition, Ashe...","[Green Day, Antonio Vivaldi, Sir Edward Elgar,...","[chillout, electro, symphony, laid back listen...","[It Must Be Love, Halo, Poison, Friday I'm in ..."
2,eartle,Michael Coffey,0,n,United Kingdom,16011,8205,0,1,"[sajones96, olivialasky, nichaoia, aCiD2, emly...",...,[],[],[],[],[],[],"[Romance Is Boring, Hold Time, My Maudlin Care...","[Morrissey, Bob Dylan, Elliott Smith, Los Camp...","[love hearts, swimming, christmas, summer, uku...","[Portions For Foxes, That's How People Grow Up..."
3,massdosage,Mass Dosage,0,n,United Kingdom,23717,17860,0,0,"[skmangal, alschuma, sven, caitlin, monkeyhack...",...,[],[],[],[],"[Erase/Rewind (Cut La Roc Vocal Mix), Rolling ...",[],"[Hamilton: An American Musical, Odd Seasons, F...","[Björk, Madlib, Flying Lotus, Beastie Boys, Ke...","[Hip-Hop, downtempo, soul, rnb, electronic, id...","[Untitled, Untitled, [untitled], Untitled, Wav..."
4,Knapster01,Matt Knapman,0,n,United Kingdom,17212,8132,0,1,"[Irina4778, ielia79, mokele, macsmax, rkapurpl...",...,[],[],[],[],[],[],"[Helioscope, Jane Doe, The Direction of Last T...","[Vessels, Portico Quartet, Fleetwood Mac, Rose...","[experimental, rock, alternative rock, christm...","[Dust (Floex Remix), Spread - Original Mix, Le..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,crstlukegates,christian gates,0,n,United Kingdom,504,320,0,0,"[alpha-ville, raffacabrone, MattSynthia, erik....",...,[],[],[],[],[],[],"[Crooks & Lovers, Causers of This, Debussy Pia...","[Valley Lines, Gordon Fergus Thompson, Mount K...",[],"[Carbonated, Snap_2, RUNNING_2_UNMASTERED, Rea..."
4996,andrius-driu,Andrius,0,n,Germany,1666,983,0,0,[alpha-ville],...,[],[],[],[],[],[],"[Veneer, The King of Limbs, Fruit, funkstörung...","[Radiohead, Bonobo, José González, Hiatus Kaiy...",[],"[Little by Little, Scuba, untitled, Burn the W..."
4997,jaylikesmusic,Jay Waldon,0,n,United Kingdom,2602,1899,0,0,"[alpha-ville, SignalPodcast, Watters89, Miss_M...",...,[],[],[],[],[],[],"[The Physical World, I Forget Where We Were, A...","[John Frusciante, Bonobo, Death from Above 197...",[],"[Right On, Frankenstein!, Virgins, Always On, ..."
4998,punct13,Punct,0,n,France,14864,8629,0,0,"[alpha-ville, pintailmusic, blond_non_blond, N...",...,[],[],[],[],[],[],"[Indiamore, Divine, Bach Cantatas for the comp...","[Gonzales, Chassol, Alexandre Tharaud, Metrono...",[castra],"[Come, ye sons of art, Music for the funeral o..."
